In [ ]:
try:
    from google.colab import drive
    
    print('Colab environment detected. Mounting drive...')
    drive.mount('/content/drive')

    print('Mounted. Switching to directory...')
    %cd /content/drive/'My Drive'/CILroadseg
except:
    print('Colab environment not found. Working on ordinary directory.')

In [ ]:
import numpy as np
import os
import matplotlib.image as mpimg

from util.helpers import *
from util.visualize import *
from util.cross_validation import cross_validate

## Loading Training and Test Data

In [ ]:
image_dir = "training/images/"
gt_dir = "training/groundtruth/"
test_dir = "test/images/"

print("Loading training input... ", end = '')
X = [np.asarray(load_image((image_dir + file))) for file in os.listdir(image_dir)]
print("done (" + str(len(X)) + " images).")
X = np.array(X)

print("Loading training groundtruth... ", end = '')
Y = [np.asarray(load_image((gt_dir + file))) for file in os.listdir(gt_dir)]
print("done (" + str(len(Y)) + " images).")
Y = (np.array(Y) >= 0.21) * 1 # compensates for lossy image data

print("Loading test input... ", end = '')
X_test = [np.asarray(load_image((test_dir + file))) for file in os.listdir(test_dir)]
print("done (" + str(len(X_test)) + " images).")
X_test = np.array(X_test)


print()
print(X.shape)
print(Y.shape)
print(X_test.shape)

## Decomposer + ResNet

In [ ]:
import tensorflow as tf
from tensorflow import keras

from decomposer import *
from resnet import *

In [ ]:
model = Decomposer(ResnetModel())
model.initialize()

# cross_validate(model, 4, images, labeled_images)

In [ ]:
np.random.seed(1) #fix randomness

model.train(Y, X)

In [ ]:
weights_path = "saves/final/resnet.h5"

print("[Target file: "+ weights_path +"]")
print("Saving model to disk...", end = '')
model.save(weights_path)
print("done.")

In [ ]:
Y_pred = model.classify(X_test)

view_image_mask(X_test[4], Y_pred[4])